In [1]:
from torchvision.models import ResNet50_Weights, Swin_T_Weights, ViT_B_16_Weights
from torch.hub import load_state_dict_from_url

from openood.evaluation_api import Evaluator
from openood.networks import ResNet50, Swin_T, ViT_B_16

from openood.networks.conf_branch_net import ConfBranchNet
from openood.networks.godin_net import GodinNet
from openood.networks.rot_net import RotNet
from openood.networks.cider_net import CIDERNet

/home/jz288/anaconda3/envs/testood/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load pre-trained model

In [2]:
arch = 'resnet50' # ['resnet50', 'swin-t', 'vit-b-16']
tvs_version = 1   # we are using torchvison V1 weights

if arch == 'resnet50':
    net = ResNet50()
    weights = eval(f'ResNet50_Weights.IMAGENET1K_V{tvs_version}')
    net.load_state_dict(load_state_dict_from_url(weights.url))
    preprocessor = weights.transforms()
elif arch == 'swin-t':
    net = Swin_T()
    weights = eval(f'Swin_T_Weights.IMAGENET1K_V{tvs_version}')
    net.load_state_dict(load_state_dict_from_url(weights.url))
    preprocessor = weights.transforms()
elif arch == 'vit-b-16':
    net = ViT_B_16()
    weights = eval(f'ViT_B_16_Weights.IMAGENET1K_V{tvs_version}')
    net.load_state_dict(load_state_dict_from_url(weights.url))
    preprocessor = weights.transforms()

net.cuda()
net.eval();

# Initialize an evaluator instance

In [3]:
# ['openmax', 'msp', 'temp_scaling', 'odin', 'mds', 'mds_ensemble', 'rmds',  
#  'gram', 'ebo', 'gradnorm', 'react', 'mls', 'klm', 'vim', 'knn', 'dice', 
#  'rankfeat', 'ash', 'she']
# it also supports some other postprocessors which need dedicated model wrappers
# see the full list in scripts/eval_ood.py and scripts/eval_ood_imagenet.py
postprocessor_name = 'ash'

# the evaluator will automatically download the required datasets
# APS (automatic hyperparameter search) will also be performed if necessary
evaluator = Evaluator(
    net,
    id_name='imagenet',                    # the target ID dataset
    data_root='./data',            # change if necessary
    config_root='./configs',       # change if necessary
    preprocessor=preprocessor,             # default preprocessing
    postprocessor_name=postprocessor_name, # the postprocessor to use
    postprocessor=None,                    # if you want to use your own postprocessor
    batch_size=200,                        # for certain methods the results can be slightly affected by batch size
    shuffle=False,
    num_workers=8)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1kXO8ESkF2GemrycdWM0sFCKH31731GPp
From (redirected): https://drive.google.com/uc?id=1kXO8ESkF2GemrycdWM0sFCKH31731GPp&confirm=t&uuid=bb3b4ef2-04ca-49f5-bb93-0ca9bf5091dc
To: /home/jz288/dev_openood/OpenOOD/test_root/data/benchmark_imglist.zip
100%|██████████| 28.0M/28.0M [00:00<00:00, 38.8MB/s]


imagenet_1k needs download:
test_root/data/images_largescale/imagenet_1k


Downloading...
From (uriginal): https://drive.google.com/uc?id=1i1ipLDFARR-JZ9argXd2-0a6DXwVhXEj
From (redirected): https://drive.google.com/uc?id=1i1ipLDFARR-JZ9argXd2-0a6DXwVhXEj&confirm=t&uuid=b520bd57-309a-452c-bc71-8c9d3a389c8e
To: /home/jz288/dev_openood/OpenOOD/test_root/data/images_largescale/imagenet_1k/imagenet_1k.zip
100%|██████████| 6.67G/6.67G [01:19<00:00, 83.8MB/s]


species_sub needs download:
test_root/data/images_largescale/species_sub


Downloading...
From (uriginal): https://drive.google.com/uc?id=1-JCxDx__iFMExkYRMylnGJYTPvyuX6aq
From (redirected): https://drive.google.com/uc?id=1-JCxDx__iFMExkYRMylnGJYTPvyuX6aq&confirm=t&uuid=a48d601c-f209-459a-bdda-9d593363efd0
To: /home/jz288/dev_openood/OpenOOD/test_root/data/images_largescale/species_sub/species_sub.zip
100%|██████████| 1.23G/1.23G [00:34<00:00, 35.8MB/s]


...

Hyperparam: [85], auroc: 0.9315319909245603


100%|██████████| 9/9 [00:04<00:00,  1.84it/s]


Hyperparam: [90], auroc: 0.9323720930232557


100%|██████████| 9/9 [00:04<00:00,  1.86it/s]

Hyperparam: [95], auroc: 0.9285631310266591
Final hyperparam: 90


# Run evaluation

In [4]:
# standard OOD detection
# the returned metrics is a dataframe
metrics = evaluator.eval_ood(fsood=False)
print(metrics)

Performing inference on imagenet test set...


100%|██████████| 225/225 [01:05<00:00,  3.46it/s]

Processing near ood...
Performing inference on species dataset...



100%|██████████| 50/50 [00:16<00:00,  3.10it/s]


Computing metrics on species dataset...
FPR@95: 67.55, AUROC: 79.53 AUPR_IN: 93.96, AUPR_OUT: 50.95
CCR: 0.27, 2.94, 9.98, 41.14, ──────────────────────────────────────────────────────────────────────

Performing inference on imagenet_21k dataset...


...

Computing metrics on sun dataset...
FPR@95: 32.48, AUROC: 93.29 AUPR_IN: 98.24, AUPR_OUT: 79.28
CCR: 1.12, 6.23, 37.02, 66.40, ──────────────────────────────────────────────────────────────────────

Performing inference on openimage_o dataset...


100%|██████████| 80/80 [00:24<00:00,  3.29it/s]

Computing metrics on openimage_o dataset...
FPR@95: 34.51, AUROC: 93.26 AUPR_IN: 97.45, AUPR_OUT: 84.00
CCR: 10.31, 18.47, 42.67, 66.16, ──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 28.55, AUROC: 94.20 AUPR_IN: 98.41, AUPR_OUT: 81.67
CCR: 5.54, 18.32, 43.40, 67.57, ──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 225/225 [01:02<00:00,  3.59it/s]

              FPR@95  AUROC  AUPR_IN  AUPR_OUT  CCR_4  CCR_3  CCR_2  CCR_1   
species        67.55  79.53    93.96     50.95   0.27   2.94   9.98  41.14  \
imagenet_21k   83.19  72.89    72.26     72.10   0.25   1.96   9.55  34.92   
nearood        75.37  76.21    83.11     61.53   0.26   2.45   9.77  38.03   
inaturalist    15.83  97.07    99.30     89.29   5.27  28.98  58.67  71.78   
texture        16.26  96.90    99.60     84.12  10.41  31.38  51.96  71.82   
places         43.66  90.48    97.46     71.63   0.59   6.53  26.71  61.69   
sun            32.48  93.29    98.24     79.28   1.12   6.23  37.02  66.40   
openimage_o    34.51  93.26    97.45     84.00  10.31  18.47  42.67  66.16   
farood         28.55  94.20    98.41     81.67   5.54  18.32  43.40  67.57   

               ACC  
species      76.18  
imagenet_21k 76.18  
nearood      76.18  
inaturalist  76.18  
texture      76.18  
places       76.18  
sun          76.18  
openimage_o  76.18  
farood       76.18  
         

In [5]:
# full-spectrum OOD detection
# would be much faster if you have ran the standard OOD detection
# since the OOD scores can be reused
fs_metrics = evaluator.eval_ood(fsood=True)

Performing inference on imagenet (cs) test set [1]: imagenet_v2...


100%|██████████| 50/50 [00:16<00:00,  3.07it/s]

Performing inference on imagenet (cs) test set [2]: imagenet_c...



100%|██████████| 50/50 [00:15<00:00,  3.29it/s]


Performing inference on imagenet (cs) test set [3]: imagenet_r...


100%|██████████| 150/150 [00:43<00:00,  3.41it/s]

Processing near ood...


Inference has been performed on species dataset...
Computing metrics on species dataset...
FPR@95: 90.09, AUROC: 63.25 AUPR_IN: 94.35, AUPR_OUT: 13.69
CCR: 0.14, 1.64, 5.66, 25.14, ──────────────────────────────────────────────────────────────────────

Inference has been performed on imagenet_21k dataset...
Computing metrics on imagenet_21k dataset...
FPR@95: 95.70, AUROC: 54.66 AUPR_IN: 73.65, AUPR_OUT: 34.93
CCR: 0.13, 1.08, 5.41, 21.02, ──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 92.90, AUROC: 58.96 AUPR_IN: 84.00, AUPR_OUT: 24.31
CCR: 0.14, 1.36, 5.53, 23.08, ──────────────────────────────────────────────────────────────────────

Processing far ood...
Inference has been performed on inaturalist dataset...
Computing metrics on inaturalist dataset...
FPR@95: 54.58, AUROC: 89.23 AUPR_IN: 98.72, AUPR_OUT: 43.79
CCR: 2.95, 17.18, 37.87, 49.46, ──────────────────────────────────────────────────────────────────────

Inference h

CSID imagenet_r Acc Eval: 100%|██████████| 150/150 [00:42<00:00,  3.53it/s]

              FPR@95  AUROC  AUPR_IN  AUPR_OUT  CCR_4  CCR_3  CCR_2  CCR_1   
species        90.09  63.25    94.35     13.69   0.14   1.64   5.66  25.14  \
imagenet_21k   95.70  54.66    73.65     34.93   0.13   1.08   5.41  21.02   
nearood        92.90  58.96    84.00     24.31   0.14   1.36   5.53  23.08   
inaturalist    54.58  89.23    98.72     43.79   2.95  17.18  37.87  49.46   
texture        50.47  89.53    99.33     34.78   5.92  18.73  32.82  49.50   
places         80.18  77.16    96.93     23.06   0.31   3.67  15.73  40.26   
sun            71.62  82.02    97.66     30.74   0.60   3.50  22.39  44.23   
openimage_o    73.62  81.47    96.39     39.33   5.86  10.63  26.20  44.02   
farood         66.09  83.88    97.81     34.34   3.13  10.74  27.00  45.49   

               ACC  
species      54.35  
imagenet_21k 54.35  
nearood      54.35  
inaturalist  54.35  
texture      54.35  
places       54.35  
sun          54.35  
openimage_o  54.35  
farood       54.35  


In [9]:
# the metrics and OOD scores are available as attributes
print(evaluator.metrics.keys())
print(evaluator.scores.keys())
print(evaluator.scores['ood']['near'].keys())
print(evaluator.scores['ood']['far'].keys())

dict_keys(['id_acc', 'csid_acc', 'ood', 'fsood'])
dict_keys(['id', 'csid', 'ood', 'id_preds', 'id_labels', 'csid_preds', 'csid_labels'])
dict_keys(['species', 'imagenet_21k'])
dict_keys(['inaturalist', 'texture', 'places', 'sun', 'openimage_o'])
